In [ ]:
%matplotlib inline

import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
class ConnectomeTransformer:
    
    def __init__(self, n_hidden, alpha = 0.001, l1_ratio = 0.3, lambd = 0.5):
        
        tf.reset_default_graph()
        
        self.n_hidden = n_hidden
        self.alpha = alpha
        self.l1_ratio = l1_ratio
        self.lambd = lambd
    
    def create_placeholders(self, data, target):
        """
        Create placeholders for data, assuming that the data is provided
        """
        
        self.X = tf.placeholder(name = "X", shape=data.shape, dtype=tf.float32)
        self.target = tf.placeholder(name = "target", shape = target.shape, dtype=tf.float32)

    def initialize_parameters(self):
        
        self.parameters = {}
        
        # Autoencoder
        self.parameters['W0'] = tf.get_variable(
            "W0", 
            [self.X.shape[1], self.n_hidden], 
            initializer = tf.contrib.layers.xavier_initializer(seed = 1), 
            regularizer = tf.contrib.layers.l2_regularizer(scale=self.alpha, scope=None)   
        )
        self.parameters['b0'] = tf.get_variable(
            "b0", 
            [self.X.shape[0],1], 
            initializer = tf.zeros_initializer()
        )
        self.parameters['b1'] = tf.get_variable(
            "b1", 
            [self.X.shape[0], 1], 
            initializer = tf.zeros_initializer()
        )
        
    def forward(self):
        
        # Autoencoder
        self.parameters['A0'] = tf.matmul(self.X, self.parameters['W0']) + self.parameters['b0']
        self.X_prime = tf.matmul(self.parameters['A0'], tf.transpose(self.parameters['W0'])) + self.parameters['b1']

    def compute_cost(self):
        
        l1_regularizer = tf.contrib.layers.l1_regularizer(
           scale=self.alpha, scope=None
        )
        
        l2_regularizer = tf.contrib.layers.l2_regularizer(
           scale=self.alpha, scope=None
        )
        
        weights = [v for v in tf.trainable_variables() if 'b' not in v.name] # all vars of your graph
        
        l1_penalty = tf.contrib.layers.apply_regularization(l1_regularizer, weights)
        l2_penalty = tf.contrib.layers.apply_regularization(l2_regularizer, weights)
        
        reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
        
        self.cost_ = tf.losses.mean_squared_error(self.target, self.X_prime)
        
    def fit(self, data = None,
            target = None,
            n_epochs = 10, 
            learning_rate = 0.0001,
            minibatch_size = 100):
        
        assert data is not None, "You need to provide data."
        
        tf.reset_default_graph()
        
        self.create_placeholders(data, target)
        self.initialize_parameters()
        
        self.forward()
        self.compute_cost()
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost_)
        
        init = tf.global_variables_initializer()
        
        tf.set_random_seed(1)
        with tf.Session() as sess:
            
            sess.run(init)
            for epoch in range(n_epochs):
                _ , cost = sess.run([optimizer, self.cost_], feed_dict={self.X: data, self.target: target})
                if epoch % 100 == 0:
                    print("Epoch %d, cost: %.5f" % (epoch, cost))
            
            self.coef_ = {}
            self.coef_['W0'] = self.parameters['W0'].eval()
            
            self.X_prime = self.X_prime.eval(feed_dict={self.X: data})
            
            """
            self.predicted_ = tf.argmax(tf.nn.softmax(self.parameters['pred_labeled']), axis=1).eval(feed_dict={
                                                                       self.X_labeled: data_labeled,
                                                                       self.Y: y})
            
            correct_prediction = tf.equal(tf.argmax(self.parameters['pred_labeled'], axis = 1), self.Y)
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            print ("Train Accuracy:", accuracy.eval({self.X_labeled: data_labeled, self.Y: y}))
            if data_test is not None and y_test is not None:
                print ("Test Accuracy:", accuracy.eval({self.X_labeled: data_test, self.Y: y_test}))
            """

In [ ]:
sc33 = np.loadtxt('../scratch/hcp_l2k8_data/100307/HCP_SC_avg_sc33.txt')
np.fill_diagonal(sc33, 0)
sc33 = (sc33 + sc33.T)/2
fc33 = np.loadtxt('../scratch/hcp_l2k8_data/100307/HCP_FC_avg_sc33.txt')

In [ ]:
plt.matshow(sc33)

In [ ]:
ae1 = ConnectomeTransformer(n_hidden=50)
ae1.fit(data = sc33, target = fc33, n_epochs = 20000)

In [ ]:
fig = plt.figure(figsize = (15,15))

ax = fig.add_subplot(221)
ax.matshow(sc33)

ax = fig.add_subplot(222)
ax.matshow(fc33)

ax = fig.add_subplot(223)
ax.matshow(ae1.X_prime)

ax = fig.add_subplot(224)
ax.scatter(ae1.X_prime, fc33)

In [ ]:
np.savetxt('../scratch/')

In [ ]:
np.corrcoef(fc33.ravel(), ae1.X_prime.ravel())